In [19]:
import sys
import os
import numpy as np
from cntk import Trainer, Axis
from cntk.io import MinibatchSource, CTFDeserializer, StreamDef, StreamDefs,\
        INFINITELY_REPEAT
from cntk.learners import sgd, learning_rate_schedule, UnitType
from cntk import input_variable, cross_entropy_with_softmax, \
        classification_error, sequence
from cntk.logging import ProgressPrinter
from cntk.layers import Sequential, Embedding, Recurrence, LSTM, Dense

In [2]:
def read_data():
    train_p = "data/20ng-train-all-terms.txt"
    test_p = "data/20ng-test-all-terms.txt"

    f_train = open(train_p)
    f_test = open(test_p)
    
    train = f_train.readlines()
    test = f_test.readlines()
    
    f_train.close()
    f_test.close()
    return train, test

In [29]:
train, test = read_data()
features, label = make_dataset(train, test)
label = np.array(label)
features = np.array(features)

In [24]:
def make_dataset(train, test):
    label = []
    features = []
    for n, line in enumerate(train, 1):
        l = line.partition('\t')
        label.append(l[0])
        features.append(l[2].split(' '))
    test_y = []
    test_x = []
    for n, line in enumerate(test, 1):
        l = line.partition('\t')
        test_y.append(l[0])
        test_x.append(l[2].split(' '))
    return features, label

In [30]:
label[0]

'alt.atheism'

In [4]:
def create_reader(path, is_training, input_dim, label_dim):
    return MinibatchSource(CTFDeserializer(path, StreamDefs(
        features=StreamDef(field='x', shape=input_dim, is_sparse=True),
        labels=StreamDef(field='y', shape=label_dim, is_sparse=False)
        )), randomize=is_training,
        max_sweeps=INFINITELY_REPEAT if is_training else 1)

In [5]:
# Defines the LSTM model for classifying sequences
def LSTM_sequence_classifier_net(input, num_output_classes, embedding_dim,
                                LSTM_dim, cell_dim):
    lstm_classifier = Sequential([Embedding(embedding_dim),
                                  Recurrence(LSTM(LSTM_dim, cell_dim)),
                                  sequence.last,
                                  Dense(num_output_classes)])
    return lstm_classifier(input)

In [35]:
# Creates and trains a LSTM sequence classification model
def train_sequence_classifier():
    input_dim = 2000
    cell_dim = 25
    hidden_dim = 25
    embedding_dim = 50
    num_output_classes = 20

    # Input variables denoting the features and label data
    features = sequence.input_variable(shape=input_dim, is_sparse=True)
    label = input_variable(num_output_classes)
    
    # Instantiate the sequence classification model
    classifier_output = LSTM_sequence_classifier_net(
        features, num_output_classes, embedding_dim, hidden_dim, cell_dim)

    ce = cross_entropy_with_softmax(classifier_output, label)
    pe = classification_error(classifier_output, label)

    rel_path = ("../../../Tests/EndToEndTests/Text/" +
                "SequenceClassification/Data/Train.ctf")
    path = os.path.join(os.path.dirname(os.path.abspath(__file__)), rel_path)

    reader = create_reader(path, True, input_dim, num_output_classes)

    input_map = {
            features: reader.streams.features,
            label:    reader.streams.labels
    }

    lr_per_sample = learning_rate_schedule(0.0005, UnitType.sample)
    # Instantiate the trainer object to drive the model training
    progress_printer = ProgressPrinter(0)
    trainer = Trainer(classifier_output, (ce, pe),
                      sgd(classifier_output.parameters, lr=lr_per_sample),
                      progress_printer)

    # Get minibatches of sequences to train with and perform model training
    minibatch_size = 200

    for i in range(255):
        mb = reader.next_minibatch(minibatch_size, input_map=input_map)
        trainer.train_minibatch(mb)

    evaluation_average = float(trainer.previous_minibatch_evaluation_average)
    loss_average = float(trainer.previous_minibatch_loss_average)
    return evaluation_average, loss_average

In [37]:
__file__ = "data/20ng-test-all-terms.txt"

In [38]:
error, _ = train_sequence_classifier()

<class 'cntk.variables.Variable'>


RuntimeError: error opening file '/home/vlad/work/text_clas/data/../../../Tests/EndToEndTests/Text/SequenceClassification/Data/Train.ctf': No such file or directory

[CALL STACK]
[0x7fdafbdb7a5c]                                                       + 0x52ba5c
[0x7fdafb063c1b]    fopenOrDie  (std::basic_string<wchar_t,std::char_traits<wchar_t>,std::allocator<wchar_t>> const&,  wchar_t const*) + 0x22b
[0x7fda401f3795]    Microsoft::MSR::CNTK::TextParser<float>::Initialize()::{lambda()#1}::  operator()  () const + 0x3b5
[0x7fda401f3953]    Microsoft::MSR::CNTK::TextParser<float>::  Initialize  () + 0x43
[0x7fda401fd49b]    Microsoft::MSR::CNTK::TextParser<float>::  TextParser  (std::shared_ptr<Microsoft::MSR::CNTK::CorpusDescriptor>,  Microsoft::MSR::CNTK::TextConfigHelper const&,  bool) + 0x8b
[0x7fda401eac2e]    CreateDeserializer                                 + 0x1be
[0x7fda4042a10b]    Microsoft::MSR::CNTK::CompositeDataReader::  CreateDeserializer  (Microsoft::MSR::CNTK::ConfigPar